# CRUD operations with entities

## Setup

Import [azure data tables](https://pypi.org/project/azure-data-tables/) (version 12.2.0 Updated nov.2021)
> pip install azure-data-tables

In [1]:
from azure.data.tables import TableClient, UpdateMode
from azure.core.exceptions import ResourceExistsError, HttpResponseError, ResourceNotFoundError


INSERT ENVIRONMENT VARIABLES

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
connection = os.getenv("AZURE_TABLE_STORAGE")
print(connection)
table = "test"

DefaultEndpointsProtocol=https;AccountName=cltj;AccountKey=L05Ry3adMrjgT66XX2/IXJb2v7UNNv3YYX4p8Qi04dGlGWMvz1gCW0lJ/1Rp9rKqHazX+nsAuje044km0mKtOA==;BlobEndpoint=https://cltj.blob.core.windows.net/;QueueEndpoint=https://cltj.queue.core.windows.net/;TableEndpoint=https://cltj.table.core.windows.net/;FileEndpoint=https://cltj.file.core.windows.net/;


## Functions

### The CRUD Function

##### To run the funciton needs:

- ```connection_string```   (needs to be extracted from azure storage account)
- ```table_name```          (create one in the portal, in azure storage explorer or programatically)
- ```operation```           (create, query, update, delete)
- ```entity```               (the entity that you want to handle, PS the entity needs to have "PartitionKey" and "RowKey")

In [3]:
def entity_crud(connection_string, table_name, operation, entity):
    with TableClient.from_connection_string(connection_string, table_name) as table_client:
        if operation == 'create':
            try:
                response = table_client.create_entity(entity=entity)
                return(response)
            except ResourceExistsError:
                print("Entity already exists")
        elif operation == 'query':
            try:
                queried_entity = table_client.get_entity(partition_key=entity['PartitionKey'],row_key=entity['RowKey'])
                return (queried_entity)
            except HttpResponseError as e:
                print(e.message)
        elif operation == 'update':
            try:
                response = table_client.update_entity(mode=UpdateMode.MERGE, entity=entity)
                return response
            except HttpResponseError as e:
                print(e.message)
        elif operation == 'delete':
            try:
                response = table_client.delete_entity(partition_key=entity['PartitionKey'],row_key=entity['RowKey'])
                return (response)
            except ResourceNotFoundError:
                print("Entity does not exists")

### Query multiple function

In [17]:
def list_entities(connection_string, table_name, select):
    with TableClient.from_connection_string(connection_string, table_name=table_name) as table_client:
        try:
            entities = list(table_client.list_entities(select=select))
            return entities
        except HttpResponseError as e:
                print(e.message)

## Playtime
<p>Now that we have our two functions, lets play with them</p>

### Single CRUD operations

First let us create a entity in a table. <br>
We already specified ```connection_string``` and the ```table```.<br>
So we only need to pass the ```operation``` and the ```entity```

In [7]:
my_data = {}
my_data['PartitionKey'] = "ThisIsATest"
my_data['RowKey'] = "1"
my_data['firstName'] = "Sam"
my_data['surName'] = "Harris"
my_data['age'] = 57
my_data['woman'] = False

print(my_data)

{'PartitionKey': 'ThisIsATest', 'RowKey': '1', 'firstName': 'Sam', 'surName': 'Harris', 'age': 57, 'woman': False}


In [8]:
# Create a new record in the table
response = entity_crud(connection_string=connection, table_name=table, operation='create', entity=my_data)
print(response)

{'date': datetime.datetime(2022, 2, 20, 8, 18, 58, tzinfo=datetime.timezone.utc), 'etag': 'W/"datetime\'2022-02-20T08%3A18%3A58.5715454Z\'"', 'version': '2019-02-02'}


In [9]:
# Query entity
response = entity_crud(connection_string=connection, table_name=table, operation="query", entity=my_data)
print(response)

{'PartitionKey': 'ThisIsATest', 'RowKey': '1', 'firstName': 'Sam', 'surName': 'Harris', 'age': 57, 'woman': False}


The query actually needs a partitionkey and a rowkey to do the search. As long as these keys is within the my_data entity it will manage to get a sepsific entity. PS 'PartitionKey' and 'RowKey' is case sensitive

In [12]:
# Now we will update our entity
my_data['firstName'] = "Camala"
my_data['age'] = 59
my_data['woman'] = True

print(my_data)

response = entity_crud(connection,table,operation="update", entity=my_data)
print(response)

{'PartitionKey': 'ThisIsATest', 'RowKey': '1', 'firstName': 'Camala', 'surName': 'Harris', 'age': 59, 'woman': True}
{'date': datetime.datetime(2022, 2, 20, 8, 31, 12, tzinfo=datetime.timezone.utc), 'etag': 'W/"datetime\'2022-02-20T08%3A31%3A13.1798331Z\'"', 'version': '2019-02-02'}


In [13]:
# Lastly we will delete our entity
response = entity_crud(connection,table,operation="delete", entity=my_data)
print(response)

None


As with query, delete also needs 'PartitionKey' and 'RowKey' in the my_data/entity <br>
PS! Note the the response is a ```None```. <br>
This signifies that the entity was deleted.